### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [10]:
!pip install langchain

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
!pip install langchain_groq

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="deepseek-r1-distill-llama-70b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x108ba55d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x108ba61d0>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [4]:
result

AIMessage(content='<think>\nAlright, the user has sent a greeting in English: "Hello How are you?" I need to translate this into French.\n\nFirst, "Hello" is commonly translated as "Bonjour" in French.\n\nNext, "How are you?" can be translated in a few ways depending on formality. Since this seems like a casual greeting, "Comment ça va?" is appropriate.\n\nPutting it together, the response should be "Bonjour, comment ça va?"\n\nI should make sure to keep the punctuation correct, adding a comma after "Bonjour" and a question mark at the end.\n\nDouble-checking for any possible errors or alternative translations, but this seems straightforward and accurate.\n</think>\n\nBonjour, comment ça va ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 16, 'total_tokens': 160, 'completion_time': 0.523636364, 'prompt_time': 0.00383292, 'queue_time': 0.0028280659999999997, 'total_time': 0.527469284}, 'model_name': 'deepseek-r1-distill-llama-70b', 

In [5]:
from langchain_core.output_parsers import StrOutputParser
#stroutputparer just display the output
parser=StrOutputParser()
parser.invoke(result)

'<think>\nAlright, the user has sent a greeting in English: "Hello How are you?" I need to translate this into French.\n\nFirst, "Hello" is commonly translated as "Bonjour" in French.\n\nNext, "How are you?" can be translated in a few ways depending on formality. Since this seems like a casual greeting, "Comment ça va?" is appropriate.\n\nPutting it together, the response should be "Bonjour, comment ça va?"\n\nI should make sure to keep the punctuation correct, adding a comma after "Bonjour" and a question mark at the end.\n\nDouble-checking for any possible errors or alternative translations, but this seems straightforward and accurate.\n</think>\n\nBonjour, comment ça va ?'

In [19]:
### Using LCEL- chain the components
#BY using chain rule we get the same result as parser!
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [20]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [22]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [23]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:'),
 HumanMessage(content='Hello')]

In [24]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [25]:
!pip install streamlit